# Importações necessárias

In [26]:
import requests
import pandas as pd
import sqlite3
from plyer import notification

# Seção 1: Funções de busca e manipulação de dados

In [27]:
def buscar_json_do_endpoint(url_endpoint):
    try:
        resposta = requests.get(url_endpoint)
        resposta.raise_for_status()
        return resposta.json()
    except requests.RequestException as erro:
        print(f"Falha na requisição: {erro}")
        return None

def converter_para_dataframe(dados):
    return pd.DataFrame(dados)

def expandir_colunas_regiao(dataframe):
    if 'regiao' in dataframe.columns:
        regiao_expandida = dataframe['regiao'].apply(pd.Series)
        regiao_expandida.columns = ['regiao_id', 'regiao_sigla', 'regiao_nome']
        dataframe = pd.concat([dataframe, regiao_expandida], axis=1).drop(columns=['regiao'])
    return dataframe

# Seção 2: Funções de coleta de dados por estado e município


In [28]:
def buscar_dados_municipio(sigla_estado):
    url_api = f"https://brasilapi.com.br/api/ibge/municipios/v1/{sigla_estado}?providers=dados-abertos-br,gov,wikipedia"
    dados_json = buscar_json_do_endpoint(url_api)
    
    if dados_json:
        dataframe = converter_para_dataframe(dados_json)
        dataframe['sigla'] = sigla_estado
        return dataframe
    return pd.DataFrame()

def criar_df_final(url_dados_estado):
    dados_estado = buscar_json_do_endpoint(url_dados_estado)
    if dados_estado is None:
        return None

    dataframe_estado = converter_para_dataframe(dados_estado)
    dataframe_estado = expandir_colunas_regiao(dataframe_estado)

    dataframes_municipios = []
    for sigla_estado in dataframe_estado['sigla']:
        dataframe_municipio = buscar_dados_municipio(sigla_estado)
        if not dataframe_municipio.empty:
            dataframes_municipios.append(dataframe_municipio)

    if not dataframes_municipios:
        print("Nenhum dado de município foi buscado.")
        return None

    dataframe_municipios = pd.concat(dataframes_municipios, ignore_index=True)
    dataframe_final = pd.merge(dataframe_municipios, dataframe_estado, on='sigla', how='inner')
    dataframe_final = dataframe_final.rename(columns={
        "nome_x": "municipio_nome",
        "nome_y": "estado_nome",
        "sigla": "estado_sigla",
        "id": "estado_id",
    })

    dataframe_final = dataframe_final.sort_values(by=['regiao_id', 'estado_nome'], ascending=True)
    return dataframe_final

# Seção 3: Função de coleta de dados bancários

In [29]:
def buscar_dados_bancos():
    url_api_bancos = "https://brasilapi.com.br/api/banks/v1"
    dados_json_bancos = buscar_json_do_endpoint(url_api_bancos)
    
    if dados_json_bancos:
        return converter_para_dataframe(dados_json_bancos)
    return pd.DataFrame()

# Seção 4: Função de notificação

In [30]:
def enviar_alerta(mensagem):
    notification.notify(
        title='Alerta',
        message=mensagem,
        timeout=10
    )

# Seção 5: Função para salvar DataFrames no SQLite


In [31]:
def salvar_df_no_sqlite(df, tabela_nome, conexao):
    try:
        df.to_sql(tabela_nome, conexao, if_exists='replace', index=False)
        enviar_alerta(f"Dados salvos com sucesso na tabela {tabela_nome}.")
    except Exception as e:
        enviar_alerta(f"Erro ao salvar dados na tabela {tabela_nome}: {e}")

# Seção 6: Função principal (main)

In [32]:
def main():
    url_dados_estado = "https://brasilapi.com.br/api/ibge/uf/v1"
    
    conexao = sqlite3.connect('database.db')
    
    df = criar_df_final(url_dados_estado)
    if df is not None and not df.empty:
        salvar_df_no_sqlite(df, 'estados_municipios', conexao)
        enviar_alerta("Dados dos estados e municípios processados com sucesso.")
    else:
        print("Falha ao recuperar ou processar dados dos estados e municípios.")
        enviar_alerta("Falha ao processar dados dos estados e municípios.")

    df_bancos = buscar_dados_bancos()
    if df_bancos is not None and not df_bancos.empty:
        salvar_df_no_sqlite(df_bancos, 'bancos', conexao)
        enviar_alerta("Dados dos bancos foram coletados com sucesso.")
    else:
        print("Falha ao recuperar dados dos bancos.")
        enviar_alerta("Falha ao coletar dados dos bancos.")
    
    conexao.close()

# Run

In [33]:
main()

def mostrar_tabelas(conexao):
    query_tabelas = "SELECT name FROM sqlite_master WHERE type='table';"
    try:
        tabelas = pd.read_sql_query(query_tabelas, conexao)
        if not tabelas.empty:
            for tabela in tabelas['name']:
                print(f"\nDados da tabela: {tabela}")
                query_dados = f"SELECT * FROM {tabela};"
                dados = pd.read_sql_query(query_dados, conexao)
                display(dados)
        else:
            enviar_alerta("Nenhuma tabela encontrada no banco de dados.")
    except Exception as e:
        print(f"Erro ao consultar tabelas: {e}")


conexao = sqlite3.connect('database.db')
mostrar_tabelas(conexao)
conexao.close()


Dados da tabela: estados_municipios


,municipio_nome,codigo_ibge,estado_sigla,estado_id,estado_nome,regiao_id,regiao_sigla,regiao_nome
0,ACRELANDIA,1200013,AC,12,Acre,1,N,Norte
1,ASSIS BRASIL,1200054,AC,12,Acre,1,N,Norte
2,BRASILEIA,1200104,AC,12,Acre,1,N,Norte
3,BUJARI,1200138,AC,12,Acre,1,N,Norte
4,CAPIXABA,1200179,AC,12,Acre,1,N,Norte
...,...,...,...,...,...,...,...,...
5565,TACURU,5007950,MS,50,Mato Grosso do Sul,5,CO,Centro-Oeste
5566,TAQUARUSSU,5007976,MS,50,Mato Grosso do Sul,5,CO,Centro-Oeste
5567,TERENOS,5008008,MS,50,Mato Grosso do Sul,5,CO,Centro-Oeste
5568,TRES LAGOAS,5008305,MS,50,Mato Grosso do Sul,5,CO,Centro-Oeste



Dados da tabela: bancos


,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,Selic,NaN,Banco Central do Brasil - Selic
3,00038166,Bacen,NaN,Banco Central do Brasil
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."
...,...,...,...,...
354,92875780,WARREN CVMC LTDA,371.0,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...
355,92894922,BANCO ORIGINAL,212.0,Banco Original S.A.
356,94968518,EFX CC LTDA.,289.0,EFX CORRETORA DE CÂMBIO LTDA.
357,,None,NaN,None
